**Thinking1: ALS都有哪些应用场景?**

- 稀疏矩阵的矩阵分解。
- 矩阵维度大，需要并行化处理。

**Thinking2：ALS进行矩阵分解的时候，为什么可以并行化处理？**

迭代公式:
1. 固定Y优化X：$x_u=(YY^T+\lambda I)^{-1}YR^T_u$
2. 固定X优化Y: $y_i=(XX^T+\lambda I)^{-1}XR_i$ 

从公式可以发现，计算单独一列$x_u$时，只和矩阵中的对应列有关$R_u$

**Thinking3: 梯度下降法中的批量梯度下降（BGD），随机梯度下降（SGD），和小批量梯度下降有什么区别（MBGD）**

- BGD：对所有样本求解梯度，梯度方向准确，但计算成本高。
- SGD：随机选择一个样本求解梯度，梯度方向不准确，但计算很快。
- MBGD：折中方法，选取一批样本求解梯度，兼顾梯度方向准确性和计算成本。

**Thinking4：你阅读过和推荐系统/计算广告/预测相关的论文么？有哪些论文是你比较推荐的，可以分享到微信群中**

本身不是这个专业的，所以没有阅读过，不过十分感兴趣，如果有好的论文会去阅读。

**Action1：对MovieLens数据集进行评分预测  工具：可以使用Surprise或者其他  说明使用的模型，及简要原理**

**读取数据**

In [25]:
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import KFold

# 数据读取
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('./L4-code/MovieLens/ratings.csv', reader=reader)
train_set = data.build_full_trainset()

**用ALS进行BaselineOnly算法**

In [27]:
from surprise import BaselineOnly
# 定义K折交叉验证迭代器, K=3
kf = KFold(n_splits=3)
# 存储K个模型
algos = []

# BaselineOnly模型
# ALS优化
bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}
# SGD优化
#bsl_options = {'method': 'sgd','n_epochs': 5}
for trainset, testset in kf.split(data):
    algo = BaselineOnly(bsl_options=bsl_options)
    #algo = NormalPredictor()
    algos.append(algo)
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

# 需要预测的数据
uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
preds = [algo.predict(uid, iid, r_ui=4, verbose=True).est for algo in algos]
print('K-fold mean predictions:', sum(preds)/len(preds))

Estimating biases using als...
RMSE: 0.8633
Estimating biases using als...
RMSE: 0.8631
Estimating biases using als...
RMSE: 0.8646
user: 196        item: 302        r_ui = 4.00   est = 4.00   {'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 4.20   {'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 4.20   {'was_impossible': False}
K-fold mean predictions: 4.1331293669449245


**用SGD进行BaselineOnly算法**

In [28]:
from surprise import BaselineOnly
# 定义K折交叉验证迭代器, K=3
kf = KFold(n_splits=3)
# 存储K个模型
algos = []

# BaselineOnly模型
# ALS优化
# bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}
# SGD优化
bsl_options = {'method': 'sgd','n_epochs': 5}
for trainset, testset in kf.split(data):
    algo = BaselineOnly(bsl_options=bsl_options)
    #algo = NormalPredictor()
    algos.append(algo)
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

# 需要预测的数据
uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
preds = [algo.predict(uid, iid, r_ui=4, verbose=True).est for algo in algos]
print('K-fold mean predictions:', sum(preds)/len(preds))

Estimating biases using sgd...
RMSE: 0.8743
Estimating biases using sgd...
RMSE: 0.8735
Estimating biases using sgd...
RMSE: 0.8758
user: 196        item: 302        r_ui = 4.00   est = 3.86   {'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 4.04   {'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 3.99   {'was_impossible': False}
K-fold mean predictions: 3.964278285335246


**可以看到在本次测试中，SGD的准确度比ALS要低**

**NomalPredictor算法**

In [29]:
from surprise import NormalPredictor
# 定义K折交叉验证迭代器, K=3
kf = KFold(n_splits=3)
# 存储K个模型
algos = []

for trainset, testset in kf.split(data):
    algo = NormalPredictor()
    algos.append(algo)
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

# 需要预测的数据
uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
preds = [algo.predict(uid, iid, r_ui=4, verbose=True).est for algo in algos]
print('K-fold mean predictions:', sum(preds)/len(preds))

RMSE: 1.4322
RMSE: 1.4316
RMSE: 1.4332
user: 196        item: 302        r_ui = 4.00   est = 5.00   {'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 5.00   {'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 2.82   {'was_impossible': False}
K-fold mean predictions: 4.27334836259108


**KNNBasic算法**

In [30]:
from surprise import KNNBasic
# 定义K折交叉验证迭代器, K=3
kf = KFold(n_splits=3)
# 存储K个模型
algos = []

for trainset, testset in kf.split(data):
    algo = KNNBasic() #use default setting
    algos.append(algo)
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

# 需要预测的数据
uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
preds = [algo.predict(uid, iid, r_ui=4, verbose=True).est for algo in algos]
print('K-fold mean predictions:', sum(preds)/len(preds))

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9026
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9052
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9038
user: 196        item: 302        r_ui = 4.00   est = 3.94   {'actual_k': 40, 'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 3.84   {'actual_k': 40, 'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 3.97   {'actual_k': 27, 'was_impossible': False}
K-fold mean predictions: 3.9170069999300323


**KNN非常占内存，而且非常耗时**

**KNNBasic with item based**

In [31]:
from surprise import KNNBasic
# 定义K折交叉验证迭代器, K=3
kf = KFold(n_splits=3)
# 存储K个模型
algos = []

sim_options = {'name': 'cosine',
               'user_based': False  # compute  similarities between items
               }
for trainset, testset in kf.split(data):
    algo = KNNBasic(sim_options=sim_options)
    algos.append(algo)
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

# 需要预测的数据
uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
preds = [algo.predict(uid, iid, r_ui=4, verbose=True).est for algo in algos]
print('K-fold mean predictions:', sum(preds)/len(preds))

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9618
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9612
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9612
user: 196        item: 302        r_ui = 4.00   est = 3.95   {'actual_k': 40, 'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 3.47   {'actual_k': 40, 'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 3.67   {'actual_k': 40, 'was_impossible': False}
K-fold mean predictions: 3.697036271529362


**SVD算法**

In [35]:
from surprise import SVD
# 定义K折交叉验证迭代器, K=3
kf = KFold(n_splits=3)
# 存储K个模型
algos = []

for trainset, testset in kf.split(data):
    # 低维矩阵维度100，迭代次数20
    algo = SVD(n_factors=100, n_epochs=20)
    algos.append(algo)
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

# 需要预测的数据
uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
preds = [algo.predict(uid, iid, r_ui=4, verbose=True).est for algo in algos]
print('K-fold mean predictions:', sum(preds)/len(preds))

RMSE: 0.8473
RMSE: 0.8460
RMSE: 0.8445
user: 196        item: 302        r_ui = 4.00   est = 4.25   {'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 4.28   {'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 4.09   {'was_impossible': False}
K-fold mean predictions: 4.202593266287534


**SlopeOne算法**

In [36]:
from surprise import SlopeOne
# 定义K折交叉验证迭代器, K=3
kf = KFold(n_splits=3)
# 存储K个模型
algos = []

for trainset, testset in kf.split(data):
    algo = SlopeOne()
    algos.append(algo)
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

# 需要预测的数据
uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
preds = [algo.predict(uid, iid, r_ui=4, verbose=True).est for algo in algos]
print('K-fold mean predictions:', sum(preds)/len(preds))

RMSE: 0.8675
RMSE: 0.8700
RMSE: 0.8670
user: 196        item: 302        r_ui = 4.00   est = 4.13   {'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 4.39   {'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 4.46   {'was_impossible': False}
K-fold mean predictions: 4.3260530208506855


**CoClustering算法**

In [37]:
from surprise import CoClustering
# 定义K折交叉验证迭代器, K=3
kf = KFold(n_splits=3)
# 存储K个模型
algos = []

for trainset, testset in kf.split(data):
    # 用户聚类数3， 商品聚类数3，迭代次数20
    algo = CoClustering(n_cltr_u=3, n_cltr_i=3, n_epochs=20)
    algos.append(algo)
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

# 需要预测的数据
uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
preds = [algo.predict(uid, iid, r_ui=4, verbose=True).est for algo in algos]
print('K-fold mean predictions:', sum(preds)/len(preds))

RMSE: 0.8986
RMSE: 0.8993
RMSE: 0.8925
user: 196        item: 302        r_ui = 4.00   est = 4.08   {'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 3.91   {'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 3.89   {'was_impossible': False}
K-fold mean predictions: 3.961027621013977


**Action2: Paper Reading：Slope one predictors for online rating-based collaborative filtering. Daniel Lemire and Anna Maclachlan, 2007. http://arxiv.org/abs/cs/0702144.
积累，总结笔记，自己的思考及idea**

### Slope one算法
简介：
- 优点：实现简单，计算迅速，合理准确，支持在线查询和动态更新
- 提出了三种Slope One算法

鲁棒性好的CF算法的目标：
1. 实现和维护简单
2. 实时更新
3. 高效访问
4. 对新用户所需的打分数据少
5. 准确且可解释

**衡量两种商品‘受欢迎差异’的方法：两种商品平均评分的差值。**（确实十分简单）
介绍了其他算法以做对比：
- user based
1. PER USER AVERAGE：直接以用户评分平均值作为评分
2. BIAS FROM MEAN：用户评分平均值+此商品偏离平均分的分数
- item based
3. ADJUSTED COSINE ITEMBASED：商品评分=两商品之间的相似度$\times$该用户对其他商品的评分的线性回归, 再做一下归一
- user based also memory based(计算用户相似性非常耗时，而且对新用户准确性较差）
4. The PEARSON Reference Scheme：用户评分平均值+此商品偏离平均分的归一化分数，此处的归一化系数是两用户的相关性

The SLOPE ONE Scheme：
- 两商品的偏差：$dev_{j,i}=\sum\limits_{u\in S_{j,i}(\chi)} \frac{u_j-u_i}{card(S_{j,i}(\chi)}$(**就是一个差值求和取平均**）
- 预测：$P(u)_j=\frac{1}{card(R_j)}\sum\limits_{i\in R_j}(dev_{j,i}+u_i)$
 - $R_j$: 用户评分过的商品且该商品也被其他用户评分过
 - **简单来说就是用用户评分过的商品加上两商品的偏差来预测被预测的商品。**
- 简化一下：$P^{S1}(u)_j=\bar{u}+\frac{1}{card(R_j)}\sum\limits_{i\in R_j}dev_{j,i}$

The WEIGHTED SLOPE ONE Scheme:
- 加权。不同商品被评分的次数代表了该商品在预测时所占的权重
- $P^{wS1}(u)_j=\frac{\sum_{i\in R_j}(dev_{j,i}+u_i)c_{j,i}}{\sum_{i\in R_j}c_{j,i}}$.
- $c_{j,i}=card(S_{j,i}(\chi))$ ：代表同时评价了$i,j$商品的用户数目

The BI-POLAR SLOPE ONE Scheme：
- 两极化。**将商品根据用户评分的平均分，分为喜欢和不喜欢两级。再用WEIGHTED SLOPE ONE分别在喜欢和不喜欢的商品集里做预测，再加权两个预测分数。**
- 两商品的偏差，这里不同的是只在用户都喜欢$i,j$时才计算在内：$$dev^{like}_{j,i}=\sum\limits_{u\in S^{like}_{j,i}(\chi)} \frac{u_j-u_i}{card(S^{like}_{j,i}(\chi)}$$
- $$P^{bpS1}(u)_j=\frac{\sum_{i\in R^{like}_j} p^{like}_{j,i}c^{like}_{j,i}+\sum_{i\in R^{dislike}_j} p^{dislike}_{j,i}c^{dislike}_{j,i}}{\sum_{i\in R^{like}_j} c^{like}_{j,i}+\sum_{i\in R^{dislike}_j} c^{dislike}_{j,i}}$$
- $c^{like}_{j,i}=card(S^{like}_{j,i}), c^{dislike}_{j,i}=card(S^{dislike}_{j,i})$
- **物以类聚，人以群分**
 - **对商品做了分类，同时被某一用户喜欢的商品更加相近，用相近的商品来预测更加准确.**
 - **对用户做了分类，同时喜欢\不喜欢$(i,j)$商品的用户喜好更加接近, 用相同喜好的用户来预测更加准确。**
 - **一点改进想法：既然是分类之后进行预测，不妨先对用户和商品进行聚类，再进行预测求平均。**
 
测试结果：
- 衡量标准：MAE
- WEIGHTED SLOPE ONE 比 SLOPE ONE 提升1%，BI-POLAR SLOPE ONE 提升1.5~2%
- BI-POLAR SLOPE ONE 和 Person 相当，但效率更高。

**Action3：设计你自己的句子生成器**

In [5]:
import random

# 定语从句语法
grammar = '''
前进 => 遇上 怪物  ， 掉血 。 
掉血 => 勇士 损失 血量
怪物 => 史莱姆 | 蝙蝠 | 骷髅 | 法师 | 魔龙 
血量 => 10 | 20 | 50 | 100 | 1000
'''

# 得到语法字典
def getGrammarDict(gram, linesplit = "\n", gramsplit = "=>"):
    #定义字典
    result = {}

    for line in gram.split(linesplit):
        # 去掉首尾空格后，如果为空则退出
        if not line.strip(): 
            continue
        expr, statement = line.split(gramsplit)
        result[expr.strip()] = [i.split() for i in statement.split("|")]
    #print(result)
    return result

# 生成句子
def generate(gramdict, target):
    if target not in gramdict: 
        return target
    find = random.choice(gramdict[target])
    return ''.join(generate(gramdict, t) for t in find)

gramdict = getGrammarDict(grammar)
print(generate(gramdict,"前进"))
print(generate(gramdict,"前进"))




遇上史莱姆，勇士损失1000。
遇上史莱姆，勇士损失50。
